In [50]:
import pandas as pd
import os
import requests
from dotenv import load_dotenv


In [55]:
load_dotenv()

VIVENU_API_KEY = os.getenv("VIVENU_API_KEY_TESTING")
headers = {
    "Authorization": f"Bearer {VIVENU_API_KEY}",
    "Accept": "application/json"
}

In [ ]:
import os
import requests
from datetime import datetime, timedelta



# 1️⃣ Calculate the last 7 days dynamically
end_date = datetime.utcnow().date()              # today (UTC)
start_date = end_date - timedelta(days=7)        # 7 days ago

# 2️⃣ Build the parameters for filtering
params = {
    "status": "COMPLETE",                        # only successful transactions
    "createdAt[gte]": start_date.isoformat(),
    "createdAt[lte]": end_date.isoformat(),
    "top": 100                                   # limit per request
}

# 3️⃣ Make the API call
url = "https://vivenu.dev/api/transactions"
response = requests.get(url, headers=headers, params=params)
response.raise_for_status()

data = response.json()
print(f"Found {data.get('total', 0)} transactions between {start_date} and {end_date}")

Found 3 transactions between 2025-11-04 and 2025-11-11


In [56]:
def fetch_total_transaction_volume():
    url = "https://vivenu.com/api/transactions"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    data = pd.DataFrame(data['docs'])
    return data['paymentCharge'].sum()

In [57]:
fetch_total_transaction_volume()

HTTPError: 401 Client Error: Unauthorized for url: https://vivenu.com/api/transactions

In [ ]:

def fetch_transactions(status="COMPLETE", skip=0, top=100):
    url = f"{VIVENU_BASE_URL}/transactions"
    params = {
        "status": status,
        "skip": skip,
        "top": top
    }
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json()

def fetch_all_complete_transactions():
    all_docs = []
    skip = 0
    top = 100
    while True:
        data = fetch_transactions(skip=skip, top=top)
        docs = data.get("docs", [])
        all_docs.extend(docs)
        if len(docs) < top:
            break
        skip += top
    return all_docs

def calculate_revenue_last_week(transactions):
    one_week_ago = datetime.utcnow() - timedelta(days=7)
    total_revenue = 0.0
    for tx in transactions:
        # parse createdAt into datetime object
        created_at = datetime.fromisoformat(tx.get("createdAt").replace("Z", "+00:00"))
        if created_at >= one_week_ago:
            # assume the field name is realPrice (docs say realPrice exists)  [oai_citation:2‡docs.vivenu.dev](https://docs.vivenu.dev/transactions?utm_source=chatgpt.com)
            total_revenue += tx.get("realPrice", 0.0)
    return total_revenue

def main():
    logging.info("Fetching transactions …")
    transactions = fetch_all_complete_transactions()
    revenue = calculate_revenue_last_week(transactions)
    logging.info(f"Revenue in last week from COMPLETE transactions: {revenue:.2f}")
    # here you could save to CSV, or push to database, or email the summary
    print(revenue)